In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time



In [3]:

#Cấu hình trình duyệt
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
# options = webdriver.ChromeOptions()
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")
# driver = webdriver.Chrome(options=options)

# Hàm để thu thập dữ liệu từ từng sản phẩm
def scrape_products(category_url, load_more_xpath):
    driver.get(category_url)
    products = []

    while True:
        try:
            # Đợi nút "Xem thêm sản phẩm" xuất hiện
            load_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, load_more_xpath))
            )
            # Click nút
            ActionChains(driver).move_to_element(load_more_button).click(load_more_button).perform()
            time.sleep(2)  # Đợi nội dung tải
        except:
            print("Đã tải hết sản phẩm.")
            break

    # Lấy toàn bộ sản phẩm
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Tìm khu vực chứa sản phẩm
    name = soup.find('h1')  # Giả sử tên sản phẩm nằm trong thẻ h1
    link = category_url  # Link sản phẩm
    table = soup.find('table', class_="content-list")
    content = soup.find('div', {'data-lcpr': 'prr-id-product-detail-product-content'})

    if table:
        rows = table.find_all('tr')  # Tìm tất cả các hàng (rows)
        data = {}

        # Duyệt qua từng dòng trong bảng và lấy thông tin theo cột
        for row in rows:
            cols = row.find_all('td')  # Tìm tất cả các cột trong dòng
            if len(cols) >= 2:
                label = cols[0].text.strip()  # Cột đầu tiên (label)
                value = cols[1].text.strip()  # Cột thứ hai (value)
                # Lưu giá trị vào từ điển data
                data[label] = value

        # Tạo một từ điển cho một sản phẩm
        product_data = {
            'Name': name.text.strip() if name else "Không có tên",
            'Link': link,
            'Chọn đơn vị tính': data.get('Chọn đơn vị tính', ''),
            'Danh mục': data.get('Danh mục', ''),
            'Dạng bào chế': data.get('Dạng bào chế', ''),
            'Quy cách': data.get('Quy cách', ''),
            'Thành phần': data.get('Thành phần', ''),
            'Chỉ định': data.get('Chỉ định', ''),
            'Chống chỉ định': data.get('Chống chỉ định', ''),
            'Nhà sản xuất': data.get('Nhà sản xuất', ''),
            'Nước sản xuất': data.get('Nước sản xuất', ''),
            'Xuất xứ thương hiệu': data.get('Xuất xứ thương hiệu', ''),
            'Thuốc cần kê toa': data.get('Thuốc cần kê toa', ''),
            'Đối tượng sử dụng': data.get('Đối tượng sử dụng', ''),
            'Mô tả ngắn': data.get('Mô tả ngắn', ''),
            'Content': content.text.strip() if content else "Không có nội dung"
        }

        products.append(product_data)

    else:
        print("Không tìm thấy bảng với class 'content-list'.")

    return products

# Đọc file CSV chứa các link
file_path = r"E:\Project\full_link.csv"
df = pd.read_csv(file_path)

# Giả sử các link nằm trong cột 'Link' của file CSV
links = df['Link'].tolist()  # Đọc tất cả các link trong cột 'Link'

# Duyệt qua từng link để thu thập dữ liệu
all_products = []
for link in links:
    print(f"Đang thu thập dữ liệu từ: {link}")
    try:
        # Gọi hàm và lấy dữ liệu cho từng link
        products = scrape_products(link, load_more_xpath="/html/body/div[1]/div[1]/div[2]/div[3]/div/div/div[2]/div/div[1]/div[2]/div[2]/div[2]/p[1]")
        # Thêm dữ liệu vào danh sách
        all_products.extend(products)
    except Exception as e:
        print(f"Không thể thu thập dữ liệu từ {link}. Chi tiết lỗi: {e}")
        continue  # Bỏ qua và tiếp tục link tiếp theo
    time.sleep(5)    
# Lưu dữ liệu vào CSV
if all_products:
    df = pd.DataFrame(all_products)
    df.to_csv("data.csv", mode='w', header=True, index=False, encoding='utf-8-sig')  # Lưu vào file CSV
    print("Dữ liệu đã được lưu vào 'data.csv'.")
else:
    print("Không có dữ liệu thu thập được.")

# Đóng trình duyệt
driver.quit()
print("Done!")


Đang thu thập dữ liệu từ: https://nhathuoclongchau.com.vn/thuoc/emla-5-3880.html
Đã tải hết sản phẩm.
Đang thu thập dữ liệu từ: https://nhathuoclongchau.com.vn/thuoc/xylocaine-2-1718.html
Đã tải hết sản phẩm.
Đang thu thập dữ liệu từ: https://nhathuoclongchau.com.vn/thuoc/lidocain-10-3242.html
Đã tải hết sản phẩm.
Đang thu thập dữ liệu từ: https://nhathuoclongchau.com.vn/thuoc/xuong-khop-truong-phuc-2x10-36054.html
Đã tải hết sản phẩm.
Đang thu thập dữ liệu từ: https://nhathuoclongchau.com.vn/thuoc/calcid-soft-usa-nic-pharma-10x10-22385.html
Đã tải hết sản phẩm.
Đang thu thập dữ liệu từ: https://nhathuoclongchau.com.vn/thuoc/calcium-d-spharm-10x10-33727.html
Đã tải hết sản phẩm.
Đang thu thập dữ liệu từ: https://nhathuoclongchau.com.vn/thuoc/glucosamine-stada-1500mg-sachet-30-goi-33211.html
Đã tải hết sản phẩm.
Đang thu thập dữ liệu từ: https://nhathuoclongchau.com.vn/thuoc/viartril-s-1795.html
Đã tải hết sản phẩm.
Đang thu thập dữ liệu từ: https://nhathuoclongchau.com.vn/thuoc/calcium